<a href="https://colab.research.google.com/github/tykiww/LLM_Implementations/blob/main/langchain_exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Langchain Prompt Templating Practice

Looking to create a structured prompt engine to stabilize the model persona.

For this example, I am creating a simple summarizer.

Inspo for this came from Sam Witteveen's Langchain Basics YT channel:
https://www.youtube.com/watch?v=J_0qvRt4LNk&list=PL8motc6AQftk1Bs42EW45kwYbyJ4jOdiZ

I still need to get the hang of using google's flan-t5-xxl appropriately, but I will get there.

That isn't the point of this exercise anyways.

For replication, be sure to

1) retrieve a huggingface API token and
2) load the API token to a config.yaml file in your google drive.

In [1]:
# Install packages
!pip -q install langchain huggingface_hub torch pyyaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 4.6 MB/s eta 0:00:00


### Setup

In [2]:
# connect to google drive to retrieve credentials.
import yaml
from google.colab import drive

def google_drive():
    """Mount the google drive (colab only)"""
    drive.mount('/content/drive')

def load_yaml(pathname):
    """retrieves Yaml file"""
    with open(pathname, "r") as f:
        config = yaml.load(f, Loader=yaml.FullLoader)
    return config

google_drive()
config = load_yaml("drive/MyDrive/config.yaml")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Session Info

In [3]:
# what gpu (colab)?
from torch import cuda


def gpu_info():
    """Searches for supported GPU. Ecxepts all errors"""
    try:
        device = cuda.get_device_name()
        n_gpus = cuda.device_count()
        print(device + ',', n_gpus, 'NVIDIA GPU(s) found.')
    except Exception as e:
        print('Supported NVIDIA GPU not found or encountered an error:\n', e)


gpu_info()

Supported NVIDIA GPU not found or encountered an error:
 Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx


### Modeling

I tend to like keeping things modularized, but since it's kinda weird to do that in a notebook, everything is built as methods within a class

#### <u>Model Object Setup</u>

In [4]:
import os, re
from langchain.llms import HuggingFaceHub
from langchain import PromptTemplate, FewShotPromptTemplate
from langchain.chains import LLMChain

In [5]:
class LLModelNavigator:
    def __init__(self, oai_token,hf_token, modelname, model_kwargs):
        self.hf_token = hf_token
        self.oai_token = oai_token
        self.modelname = modelname
        self.model_kwargs = model_kwargs

    def find_curly(self, text):
        """Helps to find all instances of input text using formatting"""
        pattern = r'{(.*?)}'
        matches = re.findall(pattern, text)
        return matches

    def set_tokens(self):
        """Sets the correct API token and is assigned upon model build"""
        os.environ['HUGGINGFACEHUB_API_TOKEN'] = self.hf_token
        os.environ['OPENAI_API_KEY'] = self.oai_token

    def set_template(self, prompt_template):
        """Sets the prompt template based on the input text"""
        prompt = PromptTemplate(
            input_variables=self.find_curly(prompt_template),
            template=prompt_template)
        return prompt

    def fewshot_template(self,kwargs):
        """Sets the prompt template based on an example prompt and text, input and separator"""
        split = kwargs["text"].split(kwargs["text_separator"])

        prompt = FewShotPromptTemplate(
        examples=kwargs["examples"],
        example_prompt=kwargs["example_prompt"],
        prefix=split[0],
        suffix=split[1],
        input_variables=self.find_curly(kwargs["text"]),
        example_separator=kwargs["example_separator"]
        )
        return prompt

    def build_model(self):
        """Sets the token and accesses said model through huggingface"""
        self.set_tokens()

        model = HuggingFaceHub(
            repo_id=self.modelname,
            model_kwargs=self.model_kwargs) # ,task="text-generation"

        return model

    def chain_model(self,prompt):
        self.set_tokens()

        model = HuggingFaceHub(
            repo_id=self.modelname,
            model_kwargs=self.model_kwargs) # ,task="text-generation"
        chain = LLMChain(llm=model, prompt=prompt)
        return chain

In [197]:

import os
os.environ['OPENAI_API_KEY'] = config['tokens']['openai']

#### <u>Model Object Build</u>

In [26]:
# set up model navigator
navigator = LLModelNavigator(
    oai_token = config['tokens']['openai'],
    hf_token=config['tokens']['huggingface'],
    modelname="google/flan-t5-xxl", # Replace this field with similar task model repos..
    model_kwargs={"temperature":.9,
                  "load_in_8bit":True,
                  "device_map": "auto",
                  "max_length":512

                  })


#### <u>General Templated Approach</u>

In [27]:
# create prompt in the navigator
prompt = navigator.set_template(prompt_template=
    """
    Summarize the following text:
    {answer}
    """)

# create chained model
llm = navigator.chain_model(prompt)

In [28]:
# Not the greatest performance, but will do for now.

llm.run("""
 Peter and Elizabeth took a taxi to attend the night party in the city.
 While in the party, Elizabeth collapsed and was rushed to the hospital.
 Since she was diagnosed with a brain injury, the doctor told Peter to stay besides her until she gets well.
 Therefore, Peter stayed with her at the hospital for 3 days without leaving.
        """)

'Peter stayed with Elizabeth for 3 days without leaving at the hospital.'

#### <u>FewShot Templated Approach</u>

Now this one is way more interesting.

My guess is the structure gives more concrete guidance + flan-t5-xxl probably needs to be re-tuned for more verbose conversations.

In [29]:
# First, create the list of few shot examples.
examples = [
    {"word": "happy", "antonym": "sad"},
    {"word": "tall", "antonym": "short"},
    {"word": "big", "antonym": "small"}
]

# Next, create a generic example prompt with it
example_prompt = navigator.set_template(prompt_template=
    """
    Word: {word}
    Antonym: {antonym}\n
    """)

# After, create the actual prompt using the FewShot Template
prompt_text = """
Give the antonym of every input

Word: {input}\nAntonym:
"""

In [30]:
# Here is a superhero version for fun.
# Run this instead of the cell above for a different take.

# First, create the list of few shot examples.
examples = [
    {"superhero": "Superman", "villain": "Lex Luthor"},
    {"superhero": "Batman", "villain": "Joker"},
    {"superhero": "Spider-Man", "villain": "Green Goblin"}
]

# Next, create a generic example prompt with it
example_prompt = navigator.set_template(prompt_template=
    """
    Superhero: {superhero}
    Villain: {villain}\n
    """)

# After, create the actual prompt using the FewShot Template
prompt_text = """
Give the villain of every superhero input

Superhero: {input}\nVillain:
"""

In [31]:
# Set up the prompt in the navigator
fewshot_prompt = navigator.fewshot_template(
    kwargs={
        "examples": examples,
        "example_prompt": example_prompt,
        "text": prompt_text,
        "text_separator": "\n\n",
        "example_separator": "\n\n"
    }
)

# create chained model
llm = navigator.chain_model(fewshot_prompt)


In [32]:
# not bad.
llm.run("Peter Pan").title()

'Hook'

#### <u>Chaining Templates</u>

Let's say that we want to connect with other APIs that interface with prompts.
PALchain does a great job stringing math into programmatic formats.

https://arxiv.org/pdf/2211.10435.pdf

In [ ]:
from langchain.chains import PALChain

navigator = LLModelNavigator(
    oai_token = '0',
    hf_token=config['tokens']['huggingface'],
    modelname="gpt2", # Replace this field with similar task model repos..
    model_kwargs={"temperature":.9,
                  #"load_in_8bit":True,
                  #"device_map": "auto",
                  "max_length":500

                  })

llm = navigator.build_model()
pal_chain = PALChain.from_math_prompt(llm, verbose=False)
question = "What is 1 + 45?"
pal_chain.run(question)

# if this doesn't run, use an OpenAI authtoken instead of the class above.
# There is a chance that the token length isn't supported with gpt2.
# I just ran out of free credits and I'm a bit cheap right now.